# Writing a new connection plugin

The connection class must inherit the abstract ``ConnectionPlugin`` class from ``nornir.core.connections`` and need to have the implementations of the methods ``open`` and ``close``. Assign the underlaying connetion to ``self.connection``.

For consistency the plugin name should be stored in the variable ``CONNECTION_NAME ``.

In [1]:
%cat writing_a_new_connection_plugin/__init__.py

from typing import Any, Dict, Optional

from nornir.core.configuration import Config
from nornir.core.connections import ConnectionPlugin


CONNECTION_NAME = "nornir_demo.demo"


class Demo(ConnectionPlugin):
    """
    This plugin is a demo plugin to show how to set up a new project
    """

    def open(
        self,
        hostname: Optional[str],
        username: Optional[str],
        password: Optional[str],
        port: Optional[int],
        platform: Optional[str],
        extras: Optional[Dict[str, Any]] = None,
        configuration: Optional[Config] = None,
    ) -> None:
        """
        Connect to the device and populate the attribute :attr:`connection` with
        the underlying connection
        """
        pass


    def close(self) -> None:
        """Close the connection with the device"""
        pass


The connection plugin can be registerd manually

In [2]:
from nornir.core.connections import Connections
from writing_a_new_connection_plugin import Demo, CONNECTION_NAME

Connections.register(CONNECTION_NAME, Demo)

Verify if the registration worked

In [3]:
connection = Connections.get_plugin(CONNECTION_NAME)
help(connection)

Help on class Demo in module writing_a_new_connection_plugin:

class Demo(nornir.core.connections.ConnectionPlugin)
 |  This plugin is a demo plugin to show how to set up a new project
 |  
 |  Method resolution order:
 |      Demo
 |      nornir.core.connections.ConnectionPlugin
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  close(self) -> None
 |      Close the connection with the device
 |  
 |  open(self, hostname:Union[str, NoneType], username:Union[str, NoneType], password:Union[str, NoneType], port:Union[int, NoneType], platform:Union[str, NoneType], extras:Union[Dict[str, Any], NoneType]=None, configuration:Union[nornir.core.configuration.Config, NoneType]=None) -> None
 |      Connect to the device and populate the attribute :attr:`connection` with
 |      the underlying connection
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset(

# Plugin auto registration

When connection plugins are installed as separate packages they should register them self as an entry point for the discovery. ``InitNornir()`` discovers and register all plugins under the entry point group ``nornir.plugins.connections`` automatically.

## Setuptools

If you use the setuptools you can use the ``entry_points`` argument in ``setup()`` to register the plugin for discovery. Add the following in your ``setup.py``.

```python
setup(
    ...
    entry_points={'nornir.plugins.connections': 'nornir_demo.demo = nornir_demo.connections:Demo'},
    ...
)
```

## Poetry

If you use poetry like the nornir project you have to add the following in the ``pyproject.toml`` file to register the plugin.

```python
[tool.poetry.plugins."nornir.plugins.connections"]
"nornir_demo.demo" = "nornir_demo.connections:Demo"
```

# Poetry demo setup

How to create a new project with poetry

Create a new project

```bash
$ poetry new nornir_demo

Created package nornir_demo in nornir_demo
```

```bash
$ tree -a -n nornir_demo

nornir_demo
├── README.rst
├── nornir_demo
│   └── __init__.py
├── pyproject.toml
└── tests
    ├── __init__.py
    └── test_nornir_demo.py

2 directories, 5 files
```

Create the folder ``connections`` and add a ``__init__.py``

```bash
$ cd nornir_demo
$ mkdir nornir_demo/connections
$ touch nornir_demo/connections/__init__.py
$ tree -a -n
.
├── README.rst
├── nornir_demo
│   ├── __init__.py
│   └── connections
│       └── __init__.py
├── pyproject.toml
└── tests
    ├── __init__.py
    └── test_nornir_demo.py

3 directories, 6 files
```

## Connection plugin implementation

You can place your code in the ``__init__.py`` under the created folder ``connections``.

In [4]:
%cat writing_a_new_connection_plugin/__init__.py

from typing import Any, Dict, Optional

from nornir.core.configuration import Config
from nornir.core.connections import ConnectionPlugin


CONNECTION_NAME = "nornir_demo.demo"


class Demo(ConnectionPlugin):
    """
    This plugin is a demo plugin to show how to set up a new project
    """

    def open(
        self,
        hostname: Optional[str],
        username: Optional[str],
        password: Optional[str],
        port: Optional[int],
        platform: Optional[str],
        extras: Optional[Dict[str, Any]] = None,
        configuration: Optional[Config] = None,
    ) -> None:
        """
        Connect to the device and populate the attribute :attr:`connection` with
        the underlying connection
        """
        pass


    def close(self) -> None:
        """Close the connection with the device"""
        pass


Don't forget to register the plugin in the ``pyproject.toml``

In [5]:
%cat writing_a_new_connection_plugin/pyproject.toml

[tool.poetry]
name = "nornir_demo"
version = "0.1.0"
description = "Nornir demo connection plugin"
authors = ["Jane Doe <jane@doe.loc>"]
license = "Apache-2.0"

[tool.poetry.plugins."nornir.plugins.connections"]
"nornir_demo.demo" = "nornir_demo.connections:Demo"

[tool.poetry.dependencies]
python = "^3.6"

[tool.poetry.dev-dependencies]
pytest = "^5.2"

[build-system]
requires = ["poetry>=0.12"]
build-backend = "poetry.masonry.api"


Nornir should be added as a dependency

```bash
$ poetry add nornir^3.0.0
```

## Test entry point registration

To test we need to install the new package. (If you use the setuptool you also need to install it with ``setup.py install`` or ``setup.py develop``)

```bash
$ poetry install -q
$ poetry shell -q
$ python
Python 3.6.9 (default, Nov  7 2019, 10:44:02)
[GCC 8.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import pkg_resources
>>> entry_point = pkg_resources.iter_entry_points('nornir.plugins.connections')
>>>
>>> for ep in entry_point:
...     if 'nornir_demo.demo' in str(ep):
...         print('Install successful')
...
Install successful
>>> exit()
```

## Documentation

Nornir plugins should be well documented. After the plugin is released and published add it to the list of available plugins ([nornir.tech](https://nornir.tech/nornir/plugins/)). The list is populated from the GitHub repo [
nornir.tech.src](https://github.com/nornir-automation/nornir.tech.src/blob/master/data/nornir/plugins.yaml). Please create a pull request.

```yaml
---
connections:
    - url: https://github.com/nornir-automation/nornir_demo
      name: demo
      maintainer: "[Jane Doe](https://github.com/janedoe)"
      description: |
          Project to demonstrate how to create a connection plugin
```